In [1]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
from astropy.time import Time



from bokeh.plotting import output_notebook, figure, show
from bokeh.models import HoverTool, tools,ColumnDataSource
output_notebook()

def _build_ref_wcs(ref_path):
        """Make a list of :class:`astropy.wcs.WCS` instances for each extension
        of the reference image.
        """
        wcs_list = []
        with fits.open(ref_path) as f:
            for ext in f:
                wcs = WCS(ext.header)
                wcs_list.append(wcs)
        return wcs, wcs_list

Loading BokehJS ...

In [2]:
outputfile = 'outdir/out.photo'
referenceimage = 'Try/final_drz_sci.chip0.fits'
columnfile = 'outdir/out.columns'

In [3]:
refwcs = _build_ref_wcs(referenceimage)


In [4]:
datafromfile = np.loadtxt(outputfile)


In [5]:
columns = []
with open(columnfile) as filecol:
    for line in filecol:
        columns.append(line)

In [6]:
xy = data[0:,2:4]


In [7]:
coordinate = refwcs[0].all_pix2world(xy,1)

In [8]:
#region to plot
findreg = 'stars.reg'

with open(findreg,'w') as fileone:
    for index, coor in enumerate(coordinate):
    #for index, coor in enumerate(xy):
            #text='circle({0},{1},.1") # color=blue text={2} \n'.format(ra[index],dec[index],'{'+str(ind[index])+"}")
            text='fk5; circle({0},{1},0.1") # color=blue text={2} \n'.format(coor[0],coor[1],'{'+str(index)+"}")
            #text='circle({0},{1},1) # color=blue text={2} \n'.format(coor[0],coor[1],'{'+str(index)+"}")
            fileone.write(text)

# index 737 is U18

In [15]:
u18 = datafromfile[737]
xyu18 = u18[2:4]
print(xyu18)

[515.39 125.65]


# Get VEGA

In [16]:
vegamagindex = np.arange(28,6568,13)

In [17]:
vegamagu18 = np.array([u18[i] for i in vegamagindex])


# Lista dates

In [18]:
listanames = []
dates = []
times =[]

for i in vegamagindex:
    initname = columns[i].find('Try/u')
    namefits = columns[i][initname:initname+23]
    listanames.append(namefits)
    fit = fits.open(namefits+'.fits')
    dates.append(fit[0].header['DATE-OBS'])
    times.append(fit[0].header['TIME-OBS'])
datesobs = [['{}T{}'.format(i,j)] for i,j in zip(dates,times)]

d = np.array(datesobs)
t = Time(d)



In [51]:
datadic = {'time':t.datetime64,
       'y':vegamagu18,
       'names':datesobs}


goodvegamagindex = np.where(np.array(vegamagu18) < 99)
data2 = {'time':t.datetime64[goodvegamagindex],
       'y':vegamagu18[goodvegamagindex],
       'names':d[goodvegamagindex]}

source = ColumnDataSource(data=data)
source2 = ColumnDataSource(data=data2)


p = figure(plot_width=900, plot_height=500, title='U18',active_drag='pan', active_scroll='wheel_zoom',
          x_axis_type='datetime',y_axis_label='VEGAMAG',x_axis_label='Date_Obs')


#Tool to get wavelength
hover2 = HoverTool(
        tooltips=[
            ('Date', '(@names)')
        ]
    )

        
p.add_tools(hover2)



# add a circle renderer with a size, color, and alpha
p.circle('time','y', source=source2)

p.add_tools(hover2)

show(p)